# Modeling Pipeline and AutoML
## Load data and packages

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import mlflow
import mlflow.sklearn

import h2o
from h2o.automl import H2OAutoML

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix

In [4]:
train_data = pd.read_csv('data/processed/train_data.csv')
test_data  = pd.read_csv('data/processed/test_data.csv')

In [5]:
train_data = train_data.drop(columns = ['Unnamed: 0','product_name'])
test_data = test_data.drop(columns = ['Unnamed: 0','product_name'])

In [6]:
X_train = train_data.drop(columns=['reordered','product_id','user_id'])
y_train = train_data['reordered']

X_test  = test_data.drop(columns=['reordered','product_id','user_id'])
y_test  = test_data['reordered']

## Preprocessing

In [8]:
categorical_cols = ['aisle_id', 'department_id']
numeric_cols = [col for col in X_train.columns if col not in categorical_cols]

In [9]:
# Transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
])

## Modeling

Since our model is trying to predict `reordered`, which is a binary variable, we will choose F1 score as our main metrics, which balances between precision and recall. We will also look at accuracy as well.

### Baseline Model (Random Forest)

In [11]:
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=32021))
])

In [12]:
# Log baseline results with MLFlow
with mlflow.start_run(run_name='Baseline_RandomForest'):
    rf_pipeline.fit(X_train, y_train)
    y_pred = rf_pipeline.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    mlflow.log_param('Model', 'RandomForest')
    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('accuracy', accuracy)
    mlflow.sklearn.log_model(rf_pipeline, 'baseline_rf_pipeline')

    # Confusion matrix plot
    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title('Confusion Matrix for Baseline RandomForest')
    cm_path = 'confusion_matrix_rf.png'
    plt.savefig(cm_path)
    plt.close(fig)
    mlflow.log_artifact(cm_path)
    
print('Baseline RF logged.')

2025/08/18 23:53:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/18 23:53:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Baseline RF logged.


### H2O AutoML

In [14]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "13" 2019-09-17; Java(TM) SE Runtime Environment (build 13+33); Java HotSpot(TM) 64-Bit Server VM (build 13+33, mixed mode, sharing)
  Starting server from /opt/anaconda3/envs/MLOps/lib/python3.13/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/kf/8ms7_h8j11bf86d6c5ykldrw0000gn/T/tmpf5xrsdms
  JVM stdout: /var/folders/kf/8ms7_h8j11bf86d6c5ykldrw0000gn/T/tmpf5xrsdms/h2o_quynhanhnd2402_started_from_python.out
  JVM stderr: /var/folders/kf/8ms7_h8j11bf86d6c5ykldrw0000gn/T/tmpf5xrsdms/h2o_quynhanhnd2402_started_from_python.err
  Server is running at http://127.0.0.1:54321
 successful.o H2O server at http://127.0.0.1:54321 ...
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,4 months and 22 days
H2O_cluster_name:,H2O_from_python_quynhanhnd2402_ct59tv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [15]:
train_h2o = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
test_h2o  = h2o.H2OFrame(pd.concat([X_test, y_test], axis=1))

train_h2o['reordered'] = train_h2o['reordered'].asfactor()
test_h2o['reordered']  = test_h2o['reordered'].asfactor()

train_h2o['aisle_id'] = train_h2o['aisle_id'].asfactor()
train_h2o['department_id'] = train_h2o['department_id'].asfactor()

test_h2o['aisle_id'] = test_h2o['aisle_id'].asfactor()
test_h2o['department_id'] = test_h2o['department_id'].asfactor()

train, valid = train_h2o.split_frame(ratios=[0.8], seed=32021)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [16]:
target = 'reordered'
features = numeric_cols + categorical_cols

In [17]:
aml = H2OAutoML(max_models=20, seed=32021, exclude_algos = ['StackedEnsemble', 'DeepLearning'], nfolds=0)
aml.train(x=features, y=target, training_frame=train, validation_frame=valid)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_grid_1_AutoML_1_20250818_235405_model_3


Model Summary: 
    number_of_trees
--  -----------------
    115

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.04337805393126096
RMSE: 0.2082739876491084
LogLoss: 0.1566098442784407
Mean Per-Class Error: 0.2608528071470811
AUC: 0.9059744828287675
AUCPR: 0.5406933348928543
Gini: 0.8119489656575349

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2344261389528239
       0       1      Error    Rate
-----  ------  -----  -------  ------------------
0      368610  12871  0.0337   (12871.0/381481.0)
1      12996   13637  0.488    (12996.0/26633.0)
Total  381606  26508  0.0634   (25867.0/408114.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.234426     0.513238  175
max f2                       0.134774     0.581967  235
max f0point5                 0.34656      0.575145  126
max accuracy                 0.375018     0.947365  116
max precision                0.908827     1         0
max recall                   0.00285563   1         396
max specificity              0.908827     1         0
max absolute_mcc             0.24766      0.479683  169
max min_per_class_accuracy   0.0837108    0.821462  278
max mean_per_class_accuracy  0.0850156    0.822643  277
max tns                      0.908827     381481    0
max fns                      0.908827     26629     0
max fps                      0.00134184   381481    399
max tps                      0.00285563   26633     396
max tnr                      0.908827     1         0
max fnr                      0.908827     0.99985   0
max fpr                      0.00134184   1         399
max tpr                      0.00285563   1         396

Gains/Lift Table: Avg response rate:  6.53 %, avg score:  6.53 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100021                   0.527298           13.6494     13.6494            0.89074          0.628751    0.89074                     0.628751            0.136522        0.136522                   1264.94   1264.94            0.135353
2        0.0200018                   0.426399           10.5737     12.1117            0.690027         0.47203     0.790396                    0.5504              0.105733        0.242256                   957.371   1111.17            0.237771
3        0.0300014                   0.359294           8.33205     10.8519            0.543739         0.391002    0.708184                    0.497272            0.0833177       0.325574                   733.205   985.194            0.316207
4        0.0400011                   0.311161           6.56727     9.78084            0.428571         0.334103    0.638285                    0.456482            0.0656704       0.391244                   556.727   878.084            0.375765
5        0.0500007                   0.275009           5.32065     8.88884            0.347219         0.292258    0.580074                    0.423639            0.0532047       0.444449                   432.065   788.884            0.421986
6        0.100001                    0.171227           3.64355     6.26619            0.237773         0.215434    0.408924                    0.319537            0.18218         0.626629                   264.355   526.619            0.563393
7        0.15               

#### Insights and best model

In [19]:
lb = aml.leaderboard
print(lb.head(10))

best_model = aml.leader
print(best_model)

model_id                                              auc    logloss     aucpr    mean_per_class_error      rmse        mse
XGBoost_grid_1_AutoML_1_20250818_235405_model_3  0.844614   0.183354  0.3579                  0.308417  0.223709  0.0500456
XGBoost_grid_1_AutoML_1_20250818_235405_model_2  0.844427   0.184076  0.353576                0.307824  0.224014  0.0501824
XGBoost_grid_1_AutoML_1_20250818_235405_model_4  0.833544   0.18809   0.332579                0.315052  0.226253  0.0511903
XGBoost_3_AutoML_1_20250818_235405               0.833281   0.188579  0.325574                0.312446  0.226773  0.0514258
XGBoost_grid_1_AutoML_1_20250818_235405_model_1  0.833269   0.188456  0.330874                0.31309   0.226351  0.0512346
XGBoost_grid_1_AutoML_1_20250818_235405_model_6  0.831525   0.189903  0.323041                0.30752   0.227193  0.0516165
XGBoost_2_AutoML_1_20250818_235405               0.830977   0.189128  0.328011                0.312985  0.226665  0.0513769
XGBoost_

#### Predict on test set and log results

In [21]:
preds = best_model.predict(test_h2o)
pred_labels = preds.as_data_frame()['predict']

f1 = f1_score(y_test, pred_labels)
accuracy = accuracy_score(y_test, pred_labels)

cm = confusion_matrix(y_test, pred_labels)
fig, ax = plt.subplots(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title("Confusion Matrix for H2O AutoML's best model")
cm_path = 'confusion_matrix_aml.png'
plt.savefig(cm_path)
plt.close(fig)

# Log automl's best model results with mlflow
with mlflow.start_run(run_name='H2O_AutoML_Best'):
    model_path = h2o.save_model(best_model, path='models', force=True)
    mlflow.log_artifact(model_path, 'h2o_aml_model')

    mlflow.log_param('model_type', 'H2O_AutoML_Leader')
    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('accuracy', accuracy)

    mlflow.log_artifact(cm_path)

print('H2O AutoML results logged.')

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


/opt/anaconda3/envs/MLOps/lib/python3.13/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


H2O AutoML results logged.
